# SunPy Interact

[SunPy Interact](https://github.com/mwhv2/SunPyInteract) is an experimental script that combines the interactive plotting power of [Plotly](https://plotly.com/python/) with SunPy Map and Timeseries objects. As this is an experiment, it is not guaranteed to work flawlessly. Instead, this script should serve as a thorough introduction to how Plotly can be used to provide a richer experience when inspecting and visualizing data in SunPy. The script can be freely modified and edited (please credit me).

Matt Wentzel-Long

In [ ]:
import sunpy.data.sample
import sunpy.map
from sunpy.timeseries import TimeSeries
from sunpy.net import Fido
from sunpy.net import attrs as a

from SunPyInteract import plotly_map, plotly_ts, ts_summary
import astropy.units as u

## Map Objects

The Plotly Express function [Imshow](https://plotly.com/python/imshow/) is used to create a Map plot similar to that of matplotlib. Currently, this plotting function works best with full disk maps.

Import some sample data

In [ ]:
aia171 = sunpy.map.Map(sunpy.data.sample.AIA_171_IMAGE)
aia193 = sunpy.map.Map(sunpy.data.sample.AIA_193_IMAGE)

 To display a Map object, use the `plotly_map` function. Just as in regular [Map plotting](https://docs.sunpy.org/en/stable/generated/gallery/plotting/aia_example.html),
 a clip interval should be specified to scale the image data (no units needed here). 
 Plotly scaling is different from that of matplotlib, so a similar clip_interval
 will not look the same as the SunPy plot functions. 
 
 Moving your cursor over the image will display the individual HPC coordinates of each plotted pixel.
 The menu at the top right allows you to switch between zoom, panning, and other cursor modes.

In [ ]:
plotly_map(aia171,clip_interval=(1,99))

 The contours function of Map objects is built into the `plotly_map` function as an argument. 
 Just specificy the intensity threshold (no units needed). The contours are overlayed as red lines. The hover
 info shows the HPC coordinates of the individual contour (the "trace #" in the hover info 
 refers to the set of data Plotly has plotted; so, "trace 3" means the third contour plotted). See [this page](https://plotly.com/python/figure-structure/#figures-as-trees-of-attributes) for information about Plotly's figure data structure. 
 Compare this plot to the SunPy example: [Finding contours of a map](https://docs.sunpy.org/en/stable/generated/gallery/map/map_contouring.html)

In [ ]:
plotly_map(aia193, clip_interval=(1,99), contours=50000)

You can also overlay a heliographic Stonyhurst coordinate (HGS) grid by setting `draw_grid=True`.
The central meridian (HGS zero longitude) and equator are slightly thicker than the other grid lines. 
Your cursor will snap to the coordinates of each meridian or parallel when you get near it. 
The hover info also includes heliographic Carrington coordinates (HGC). 

The individual gridlines are calculated following the method outlined in this SunPy example: [Drawing heliographic longitude and latitude lines](https://docs.sunpy.org/en/stable/generated/gallery/plotting/lat_lon_lines.html)

In [ ]:
plotly_map(aia171, clip_interval=(1,99), draw_grid=True)

The `plotly_map` function can overlay the solar limb as a blue line on a Map by setting `draw_limb=True`. 

In [ ]:
plotly_map(aia171, draw_limb=True)

It can also draw the limb from the viewpoint of another Map by setting `draw_limb` equal to the observer of the other Map. In the example below, an AIA map is downloaded from two months after the AIA_171_IMAGE sample data. The hidden part of the solar limb is represented as less opaque blue dots.

In [ ]:
q = Fido.search(a.Time('2011-08-01T00:00:00', '2011-08-01T00:00:11'),
                a.Instrument('AIA'),
                a.Wavelength(wavemin=171*u.angstrom, wavemax=171*u.angstrom))

In [ ]:
download = Fido.fetch(q)
aia171_8 = sunpy.map.Map(download)

In [ ]:
observer = aia171_8.center.observer
plotly_map(aia171, draw_limb=observer)

For maps with higher angular resolution than the sample data maps, the `resample` argument of `plotly_map` can be passed dimensions to resample the given map. This can result in a smoother interactive interface.

In [ ]:
print(aia171_8.data.shape)
plotly_map(aia171_8, draw_grid=True, resample=[2048,2048])

You can pass the `plotly_map` function a colorscale to apply to the image.
Caution, this will slow down the interactivity of the plot especially for large Maps.
Here is a link to Plotly's built-in colorscales, which can also be customized:
https://plotly.com/python/builtin-colorscales/

For example, the scale below is based on the sequential scale "solar", except the 
darkest color has been replaced with black.

In [ ]:
cscale = ['rgb(0, 0, 0)', 'rgb(79, 28, 33)', 'rgb(108, 36, 36)', 'rgb(135, 47, 32)', 
              'rgb(157, 66, 25)', 'rgb(174, 88, 20)', 'rgb(188, 111, 19)', 'rgb(199, 137, 22)', 
              'rgb(209, 164, 32)', 'rgb(217, 192, 44)', 'rgb(222, 222, 59)']
plotly_map(aia171, clip_interval=(1,99),color_scale=cscale)

There's also `summary` argument which displays an information table next to the interactive plot. 

In [ ]:
 plotly_map(aia171, clip_interval=(1,99), summary=True, draw_grid=True)

`plotly_map` also works with heliographic maps. Note: currently only pixel coordinates are provided. `draw_limb` cannot yet be used with heliographic maps. Also, `summary` does not work with heliographic maps. Compare this to: [Creating a Heliographic Map](https://docs.sunpy.org/en/stable/generated/gallery/map_transformations/reprojection_heliographic_stonyhurst.html).

In [ ]:
from astropy.coordinates import SkyCoord
shape_out = (720, 1440)
frame_out = SkyCoord(0, 0, unit=u.deg,
                     frame="heliographic_carrington",
                     observer=aia193.observer_coordinate,
                     obstime=aia193.date,
                     rsun=aia193.coordinate_frame.rsun)
header = sunpy.map.make_fitswcs_header(shape_out,
                                       frame_out,
                                       scale=(360 / shape_out[1],
                                              180 / shape_out[0]) * u.deg / u.pix,
                                       projection_code="CAR")
outmap = aia193.reproject_to(header)

In [ ]:
plotly_map(outmap)

Last, but not least, there is a `draw` argument in `plotly_map` that overlays a drawing plane over the map. This allows you draw lines, circles, boxes, and other shapes on top of the map. Set draw equal to a color. You can switch between drawing modes using the menu in the upper right corner.

In [ ]:
plotly_map(aia171, clip_interval=(1,99), draw='orange')

## TimeSeries Objects

TimeSeries data is plotted using the Plotly function [Scatter](https://plotly.com/python/line-and-scatter/#scatter-and-line-plots-with-goscatter) (specifically, this uses the Plotly [Graph Objects class](https://plotly.com/python/graph-objects/) instead of the Plotly Express module).

First, load some sample data.

In [ ]:
goes_lc = TimeSeries(sunpy.data.sample.GOES_XRS_TIMESERIES)
gbm = TimeSeries(sunpy.data.sample.GBM_TIMESERIES, source='GBMSummary')

`plotly_ts` will plot all channels of data on the same figure. 
You can toggle data by clicking on the channel name in the legend
and zoom/pan around by selecting the cursor modes in the top right.

In [ ]:
plotly_ts(goes_lc)

The `peaks` argument will find peaks for all channels in the timeseries object, given
a minimum difference value that defines a peak.
The hover info will display the time and date as well as the max/min flux values.
Compare to the following SunPy example from which the findpeaks function is taken: [Find Peaks in sunpy TimeSeries](https://docs.sunpy.org/en/stable/generated/gallery/time_series/timeseries_peak_finding.html)

In [ ]:
my_ts = goes_lc.truncate('2011/06/07 06:10', '2011/06/07 09:00')
plotly_ts(my_ts, peaks=1e-7)

The `gradient` argument of `plotly_ts` will take the NumPy gradient for each channel
in a TimeSeries. First, download some GOES 15 data.

In [ ]:
tstart = "2015-06-21 01:00"
tend = "2015-06-21 23:00"
result_goes15 = Fido.search(a.Time(tstart, tend), a.Instrument("XRS"), a.goes.SatelliteNumber(15))
file_goes15 = Fido.fetch(result_goes15)
goes_15 = TimeSeries(file_goes15)
goes_15 = goes_15.remove_column("xrsa_quality")
goes_15 = goes_15.remove_column("xrsb_quality")

Now truncate the TimeSeries around a flare, and plot the gradients.
Compare this to the following SunPy example: [Retrieving and analyzing GOES X-Ray Sensor (XRS) data](https://docs.sunpy.org/en/stable/generated/gallery/acquiring_data/goes_xrs_example.html)

In [ ]:
goes_flare = goes_15.truncate("2015-06-21 09:35", "2015-06-21 10:30")
plotly_ts(goes_flare, gradient=True)

The power spectrum of each channel can be plotted using the `power_spec` argument. This uses [Scipy's periodogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.periodogram.html) function to calculate the spectrum. Set `power_spec` equal to the sampling frequency. Compare to the following SunPy example: [Making a power spectrum from a TimeSeries](https://docs.sunpy.org/en/stable/generated/gallery/time_series/power_spectra_example.html)

In [ ]:
plotly_ts(goes_lc, power_spec=0.25)

The `ts_summary` function provides an interactive summary of the timeseries
object. This can be especially useful for inspecting multiple channels of data. 
Use the dropdown menu in the top left to switch between channels.
The histograms are binned according to Scott's rule using the Astropy function:
https://docs.astropy.org/en/stable/api/astropy.stats.scott_bin_width.html

In [ ]:
ts_summary(gbm)

## The Basic Plotting Process

In this section, a few short examples will demonstrate the basic plotting steps that `plotly_map` and `plotly_ts` are built on. For more information on the dropdown menus of `ts_summary`, see the official [Dropdown menus in Python](https://plotly.com/python/dropdowns/) page.

###  plotly_map

The code below creates a basic plot of the AIA171 sample data using `plotly.express.imshow`. Clipping bounds are passed to the `zmin` and `zmax` argmuents, `origin` specifies that the origin of the image is in the lower left corner, and `binary_string` converts the data to a b64 PNG string before being passed to the plotly.js script (see [here](https://plotly.com/python-api-reference/generated/plotly.express.imshow.html) for more details). 

`plotly_map` uses the same AsymmetricPercentileInterval function for clipping as in SunPy. To change the axes from pixels to another unit, additional arguments for `x` and `y` can be provided (e.g., as NumPy arrays that match the shape of the map). 

In [ ]:
import plotly.express as px
from astropy.visualization import AsymmetricPercentileInterval

clip_interval = (1,99)
clip_interval = clip_interval*u.percent
clip_percentages = clip_interval.to('%').value
vmin, vmax = AsymmetricPercentileInterval(*clip_percentages).get_limits(aia171.data)
px.imshow(aia171.data, origin='lower', zmin=vmin, zmax=vmax, binary_string=True, 
          labels={'x':'Helioprojective Longitude (pixels)',
                  'y':'Helioprojective Latitude (pixels)'},
          title="AIA 171 Plot")

### plotly_ts

In this example, `plotly.graph_objects.Scatter` is used to create a basic timeseries plot. `plotly.graph_objects` is the plotting backend that the `plotly.express` API uses. It allows for more customization, but requires a slightly more complicated plotting process.

First, the timeseries data is written to a Pandas DataFrame (Plotly communicates with Pandas very well). The Figure base is created next, and a for-loop plots each channel of data on the figure as a [plotly.graph_objects.Scatter](https://plotly.com/python-api-reference/generated/plotly.graph_objects.Scatter.html#plotly.graph_objects.Scatter) plot. Plotly Scatter plots will default to line plots if the density of points becomes high but can be overridden/specified. Then, two commands change the yaxis title and transform it a logarithmic axis. In the `update_layout` command, the specific formatting of the yaxis ticks (powers of ten) is updated and the [hovermode](https://plotly.com/python/hover-text-and-formatting/) is changed to the "x" format.

In [ ]:
import plotly.graph_objects as go

dat = gbm.to_dataframe()
channels = gbm.columns

fig = go.Figure()
for i in channels:
    fig.add_trace(go.Scatter(x=dat.index, y=dat[i], name=i))
fig.update_yaxes(type="log", title=gbm.units[channels[0]].to_string())
fig.update_layout(yaxis_dtick="1",yaxis_exponentformat="power",
                  hovermode="x")
fig.show()

Lastly, the example below shows the ease of use that `plotly.express` offers for creating a timeseries plot with minimal customization done to the settings. Here, [plotly.express.line](https://plotly.com/python-api-reference/generated/plotly.express.line.html#plotly.express.line) is passed the entire DataFrame defined in the previous example. 

In [ ]:
px.line(dat, log_y=True, labels={'value':gbm.units[channels[0]].to_string(),
                                 'index':'Date'}
       )